# Bidirection Compression


## 1. Implement an environment that would emulate the communication of workers with the server


In [1]:
import time
import numpy as np


class NoopServerToWorkerCompressor:
    def __init__(self):
        self.transmitted_coordinated = 0

    def compress(self, X, y):
        self.transmitted_coordinates = X.shape[0] * X.shape[1] + y.shape[0] * y.shape[1]
        return X, y

    def decompress(self, X, y):
        return X, y


class DistributedEnvSimulator:
    def __init__(
        self,
        execute_on_device,
        split_data,
        worker_to_server_compressor,
        server_to_worker_compressor=NoopServerToWorkerCompressor(),
    ):
        self.worker_to_server_compressor = worker_to_server_compressor
        self.server_to_worker_compressor = server_to_worker_compressor
        self.execute_on_device = execute_on_device
        self.split_data = split_data

    def simulate_distributed_env(self, X, y, n_devices, heterogeneousity=0, num_iterations=100, eps=None):
        n_features = X.shape[1]
        weights = np.zeros(n_features)
        convergence = []
        accuracies = []
        execution_time = 0

        X_split, y_split = self.split_data(X, y, n_devices, heterogeneousity)

        for iteration in range(num_iterations):
            start_time = time.time()
            aggregated_gradient = np.zeros(n_features)

            for device_idx in range(n_devices):
                X_device = X_split[device_idx]
                y_device = y_split[device_idx]

                compressed_gradient, indices = self.execute_on_device(
                    X_device,
                    y_device,
                    iteration,
                    n_devices,
                    n_features,
                    self.worker_to_server_compressor,
                    self.server_to_worker_compressor,
                )

                aggregated_gradient += self.worker_to_server_compressor.decompress(
                    compressed_gradient, indices, len(X_device)
                )

            weights -= aggregated_gradient / n_devices
            execution_time += time.time() - start_time

            accuracy_i = self._estimate_accuracy(X, y, weights)
            accuracies.append(accuracy_i)
            convergence_i = self._estimate_convergence(aggregated_gradient / n_devices)
            convergence.append(convergence_i)

            if eps is not None and accuracy_i < eps:
                break

        transmitted_coordinates_worker_to_server = self.worker_to_server_compressor.transmitted_coordinates
        transmitted_coordinates_server_to_worker = self.server_to_worker_compressor.transmitted_coordinates

        return (
            weights,
            convergence,
            accuracies,
            execution_time,
            transmitted_coordinates_worker_to_server,
            transmitted_coordinates_server_to_worker,
        )

    def _estimate_accuracy(self, X, y, weights):
        y_pred = np.sign(np.dot(X, weights))
        diff = y.astype("int") - y_pred.astype("int")
        false_predictions = len(diff[diff != 0])
        accuracy = 1 - false_predictions / len(y_pred)
        return accuracy

    def _estimate_convergence(self, aggregated_gradient, n_devices):
        return np.linalg.norm(aggregated_gradient / n_devices)


# def example_execute_on_device_cgd(X, y, iteration, n_devices, n_features, compression_ratio):
#     L = L = np.sum(np.linalg.vector_norm(X, axis=1) ** 2) / (4 * X.shape[1])
#     lambda_reg = L / 1000
#     gamma = gamma_k(L, omega, n_devices, iteration)

#     gradient_device = nabla_f(X_device, y_device, weights, lambda_reg)
#     compressed_gradient, indices = randk_compress(gradient_device, k)

#     return compressed_gradient, indices

## 2. Implement CGD